In [2]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [1]:
from openai import OpenAI
import numpy as np

# Keeping the original code ============================
client = OpenAI(api_key="sk-svcacct-oD2ywALJLsVNQwU3GZ-Lvl-6d53TV1hwjr7DKEfvT07SSYDzFHvmTo3Fx9gp-5KT3BlbkFJsc-MOn7QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA")

#===== Creating an embedding with the OpenAI API =======
response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)
# print(response.data[0].embedding)
# ======================================================

# ==== Use case example ======
query = "Who is the greatest basketball player of all time?"

resource1 = "Lebron james is likely the greatest NBA player of all time."
resource2 = "Messi is likely the greatest soccer player of all time"
resource3 = "RAG stands for Retrieval Augmented Generation"

query_embedding = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

resource_responses = client.embeddings.create(
            input=[resource1, resource2, resource3],
            model="text-embedding-3-small"
    ).data

resource1_embedding = resource_responses[0].embedding
resource2_embedding = resource_responses[1].embedding
resource3_embedding = resource_responses[2].embedding

query = np.array(query_embedding)

resource1_embedding = np.array(resource1_embedding)
resource2_embedding = np.array(resource2_embedding)
resource3_embedding = np.array(resource3_embedding)

def cosine_simularity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

simularity1 = cosine_simularity(query_embedding, resource1_embedding)
simularity2 = cosine_simularity(query_embedding, resource2_embedding)
simularity3 = cosine_simularity(query_embedding, resource3_embedding)

print(f"Similarity of query and resource1: {simularity1}")
print(f"Similarity of query and resource2: {simularity2}")
print(f"Similarity of query and resource3: {simularity3}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [4]:


def rag_document_answering(file_path, question):
    import openai
    import numpy as np

    openai.api_key = "sk-svcacct-oD2ywALJLsVNQwU3GZ-Lvl-6d53TV1hwjr7DKEfvT07SSYDzFHvmTo3Fx9gp-5KT3BlbkFJsc-MOn7QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA"

    # Step 1: Read the messi.txt file and split the text
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Split the text into chunks (500 characters each, 조절 가능)
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]

    # Step 2: Generate embeddings for each chunk and store them in a dictionary
    chunk_embeddings = {}
    for chunk in chunks:
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-3-small"
        )
        chunk_embeddings[chunk] = response.data[0].embedding

    # Step 3: Generate an embedding for the question
    question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding

    # Step 4: Calculate cosine similarities between the question and each chunk
    def cosine_similarity(A, B):
        return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

    similarities = {
        chunk: cosine_similarity(np.array(question_embedding), np.array(embedding))
        for chunk, embedding in chunk_embeddings.items()
    }

    # Step 5: Select the top 5 most similar chunks
    top_chunks = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:5]
    top_context = "\n".join([chunk for chunk, _ in top_chunks])

    # Step 6: Create a RAG-enhanced prompt
    rag_prompt = f"""
Using the following context, answer the question:

Context:
{top_context}

Question: {question}
Answer:
"""

    # Step 7: Use the LLM to generate an answer
    response = openai.Completion.create(
        model="gpt-4",
        prompt=rag_prompt,
        max_tokens=150,
        temperature=0.7
    )

    # Extract the response text
    answer = response.choices[0].text.strip()

    return answer

if __name__ == "__main__":
    user_question = "What are Messi's greatest achievements?"

    messi_file_path = "messi.txt"

    generated_answer = rag_document_answering(messi_file_path, user_question)

    print("\nGenerated Answer:")
    print(generated_answer)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}